## Attention机制

![](../images/scaled-dot-product-attention.png)

> - 图中`Decoder`的隐藏状态向量`query` $q_t$ 与`Encoder`的隐藏状态向量序列`key` $K=[k_1, k_2, k_3, k_4]$ 每个向量做点积，每个 `key`向量对应一个分数。
- 然后`sofmax`转换为权重分布，权重值越大可以认为越“关注”该处向量的信息。
- 每个位置的 $v$ 与该处的权重相乘，然后求和得到 $q_t$ 对 $K$ 做注意力的输出向量。

## Pytorch实现

In [1]:
import torch, math, torch.nn.functional as F


def attention(query, key, value, mask=None, dropout=None):
    """
    query : batch, target_len, feats
    key   : batch, seq_len,    feats
    value : batch, seq_len,    val_feats
    
    return: batch, target_len, val_feats
    """
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim=-1)

    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

`query`与`key`去做点积，能够得到`seq_len`个点积之后的结果。再做个`mask`和`softmax`之后，我们就可以得到`attention`，此时的`attention`的长度是$target\_len \times seq\_len$, 再与编码器的value输出纬度$seq\_len \times val\_feats$做矩阵乘法之后，就可以得到解码器`decoder`的输出：

In [2]:
def test_attention():
    query = torch.randn(3, 5, 4)  # batch, target_len, feats
    key = torch.randn(3, 6, 4)  # batch, seq_len, feats
    value = torch.randn(3, 6, 8)  # batch, seq_len, val_feats
    attn, _ = attention(query, key, value)
    print(attn.shape)
    assert attn.shape == (3, 5, 8)
    print("Test passed")

test_attention()

torch.Size([3, 5, 8])
Test passed
